<a href="https://colab.research.google.com/github/SingTown/openmv_tensorflow_training_scripts/blob/main/mnist/openmv_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train mnist and Save to OpenMV

This Code is for TensorFlow 2

reference: https://keras.io/examples/vision/mnist_convnet/

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Prepare Dataset

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Define Model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)


# Train model

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 47s 97ms/step - accuracy: 0.7613 - loss: 0.7718 - val_accuracy: 0.9772 - val_loss: 0.0815
Epoch 2/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 76s 85ms/step - accuracy: 0.9619 - loss: 0.1256 - val_accuracy: 0.9845 - val_loss: 0.0583
Epoch 3/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 41s 85ms/step - accuracy: 0.9731 - loss: 0.0881 - val_accuracy: 0.9865 - val_loss: 0.0477
Epoch 4/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 35s 84ms/step - accuracy: 0.9786 - loss: 0.0727 - val_accuracy: 0.9868 - val_loss: 0.0464
Epoch 5/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 41s 85ms/step - accuracy: 0.9801 - loss: 0.0627 - val_accuracy: 0.9900 - val_loss: 0.0384
Epoch 6/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 40s 83ms/step - accuracy: 0.9826 - loss: 0.0548 - val_accuracy: 0.9888 - val_loss: 0.0381
Epoch 7/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 41s 84ms/step - accuracy: 0.9837 - loss: 0.0492 - val_accuracy: 0.9907 - val_loss: 0.0358
Epoch 8/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 41s 84ms/step - accuracy: 0.9846 - loss: 0.0479 - 

# Evaluate the trained model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.023657817393541336
Test accuracy: 0.9919000267982483


# Full Integer Quantization
ref: https://www.tensorflow.org/lite/performance/post_training_quantization#full_integer_quantization

In [ ]:
import tensorflow as tf
from tensorflow import lite

def representative_dataset():
    for i in range(100):
        yield [np.array([x_test[i]])]

# Convert the tflite.
converter = lite.TFLiteConverter.from_keras_model(model)
converter._experimental_disable_per_channel_quantization_for_dense_layers = True
converter.optimizations = [lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()

# Save the model.
with open('trained.tflite', 'wb') as f:
  f.write(tflite_quant_model)

Saved artifact at '/tmp/tmpoo_eveu6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134643098043792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134643098045136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134643098044560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134643098048016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134643098048592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134643098048784: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


# Succeed
Copy trained.tflite to OpenMV4 H7 or OpenMV4 H7 Plus, run this code in OpenMV: https://github.com/SingTown/openmv_tensorflow_training_scripts/blob/main/mnist/main.py